In [345]:
import pickle
import pandas as pd

#GetRouteByStation 오류나는 중
with open(r"/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/getRouteByStation.pickle", 'rb') as file:
    getRouteByStation = pickle.load(file)

with open(r"/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/getStationByRoute.pickle", 'rb') as file:
    getStationByRoute = pickle.load(file)

#지하철역 인접 여부
with open(r"/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/getStationByPosList.pickle", 'rb') as file:
    getStationByPosList = pickle.load(file)



In [172]:
od = pd.read_csv('/Users/taejin/Desktop/데분 학회/급행버스/od_weekday.csv')

### 1.구간 길이 계산

In [173]:
od['구간길이'] = 0
a=0
for key in getStationByRoute.keys():
    od_df = od[od['노선명'] == key] # od 노선명 필터링 df
    dis_df = getStationByRoute[key] # 역간 거리정보 df 노선명 기준 필터링
    route_type = dis_df['routeType'].unique() #간선: 3, 지선:4
    
    # 간선 지선만 추출
    if (route_type == '3') or (route_type == '4'):
        #승차, 하차 순번 리스트
        start_seq = od_df['승차_정류장순번'].unique()
        end_seq = od_df['하차_정류장순번'].unique()
        
        print(key)
        for i in start_seq:
            for j in end_seq:
                if i<j:
                    filtered = dis_df[(dis_df['seq'].astype(int) > i) & (dis_df['seq'].astype(int) <= j)] # 거리정보를 정류장 순서 기준 필터링
                else:
                    filtered = dis_df[(dis_df['seq'].astype(int) > j) & (dis_df['seq'].astype(int) <= i)] # 거리정보를 정류장 순서 기준 필터링
                try:
                    index = od_df[(od_df['승차_정류장순번'] == i) & (od_df['하차_정류장순번'] == j)].index[0]
                    od.at[index, '구간길이'] = filtered['fullSectDist'].astype(int).sum()
                except IndexError:
                    pass
    else:
        od = od[od['노선명'] != key]


1161
1157
1130
707
N854
773
N64
N65
6632
6645
6647
6633
6642
601
N26
605
6631
422
351
3413
3212
340
N13B
6614
703
8552
7727
260
710
270
3317
8335
8331
370
3323
3318
2312A
3411
3412
342
2312
1137
N13
1129
6715
6624
5538
541
5623
5530
5531
540
5513
163
151
101
N15
753
401
301
3426
6643
7711
7726
740
750A
8551
8111
152
1226
421
463
405
4211
400
775
1221
1143
702A서오릉
702B용두초교
2013
271B
1213
271
470
6712
6641
651
672
6648
642
654
6629
N15A
8112
102
0017
N61B
2227
2115
162
143
500
1155
5617
5525
5537
N51
5621
6516
6615
6627
6657
6514
6620
6716
6640B
N61
6613
6515
643
6640A
6623
1112
4318B
7713
506
674
441
502
5624
542
111
107
462
407
8003
507
1125A
1218
1144
1154
1120
100
1227
7475
6649
1020
110A고려대
110B국민대
1113
5519
1133
N866
5618
8541
440
3420
333
452
361
343
345
N840
4318
4319
4212
0411
N877
4433
752
7720
7022
742
8773
심야A21
1017
1140
173
604
652
6650
8001
8701
471
774
5528
571
5012
8771
5626
5713
5625
600
670
N16B
660
673
661
606
144
130
121
1165
242
2234
2236
262
273
240
2016
N26A
2015


In [178]:
import copy 
temp = copy.deepcopy(od.drop('Unnamed: 0', axis =1))

def delete_rows_starting_with(df, column, letter):
    return df[~df[column].str.startswith(letter)]

temp = delete_rows_starting_with(temp, '노선명', 'N')

temp.to_csv('/Users/taejin/Desktop/데분 학회/급행버스/od_weekday_2.csv')

#### 2. 중장거리 비율, 총승객수, 총거리, 굴곡도

In [393]:
routeNms = temp['노선명'].unique()
km = 10000 #meter (중장거리 기준)

long_dis = pd.DataFrame(routeNms, columns=['노선명'])

long_dis['중장거리비율'] = 0


중장거리비율

In [394]:
# 중장거리비율 입력
for name in routeNms:
    print(name)
    df = temp[temp['노선명'] == name]
    sum_prodcut_all = df['승객수'] * df['구간길이']
    
    filtered = df[df['구간길이'] >= km]
    sum_product = filtered['승객수'] * filtered['구간길이']
    #중장거리 비율 계산
    ratio = sum_product.sum()/sum_prodcut_all.sum()
    
    #계산 값 입력
    index = long_dis[long_dis['노선명'] == name].index[0]
    long_dis.at[index, '중장거리비율'] = ratio

1130
773
0017
0411
100
101
1014
1017
102
1020
103
104
105
106
107
109
110A고려대
110B국민대
111
1111
1113
1115
1116
1119
1120
1122
1124
1126
1127
1128
1129
1131
1132
1135
1136
1137
1138
1139
1141
1142
1143
1144
1154
1155
1162
1164
1165
1167
120
121
1213
1218
1221
1222
1224
1226
1227
130
140
141
142
143
144
145
146
147
148
150
151
152
160
162
171
1711
172
173
201
2012
2013
2014
2015
2016
202
2112
2113
2114
2115
2211
2212
2221
2222
2224
2227
2230
2233
2236
2311
2312
240
241
2412
2413
2415
2416
242
260
261
262
270
271
272
273
301
3011
302
303
320
3212
3214
3216
3217
3220
3314
3315
3316
3317
3318
3319
3321
3322
3323
333
340
341
3411
3412
3413
3414
3416
3417
342
3420
3422
3426
343
345
350
360
361
370
400
401
402
405
406
420
421
4211
4212
4312
4318
4319
440
441
4425
4432
4433
4435
452
461
463
470
472
500
501
5012
502
503
504
505
506
540
541
5413
542
5511
5513
5515
5516
5517
5519
5522A난곡
5522B호암
5523
5524
5525
5528
5530
5531
5535
5536
5611
5615
5616
5618
5619
5620
5621
5623
5625
5626
5627
5630
5633

노선총거리

In [395]:
#노선 총 거리
distance_df = pd.read_excel('/Users/taejin/Desktop/데분 학회/급행버스/서울시버스노선기본정보(20240507).xlsx')
mapping_df = distance_df[['노선번호', '거리']]
mapping_dict = mapping_df.set_index(mapping_df.columns[0])[mapping_df.columns[1]].to_dict()
long_dis['노선총거리(km)'] = long_dis['노선명'].map(mapping_dict)

총승객수

In [396]:
#승객수
nms = long_dis['노선명'].unique()
long_dis['총승객수'] = 0 
for name in nms:
    df = temp[temp['노선명'] == name]
    passenger = df['승객수'].sum()
    idx = long_dis[long_dis['노선명'] == name].index[0]
    long_dis.at[idx, '총승객수'] = passenger

굴곡도

In [397]:
import math

def distance(df1, df2):
    # 위도와 경도를 라디안으로 변환
    lat1 = math.radians(float(df1['gpsY'].iloc[0]))
    lon1 = math.radians(float(df1['gpsX'].iloc[0]))
    lat2 = math.radians(float(df2['gpsY'].iloc[0]))
    lon2 = math.radians(float(df2['gpsX'].iloc[0]))
    
    # 하버사인 공식 적용
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2.0)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 지구 반경 (단위: km)
    R = 6370
    distance = R * c
    
    return distance


In [398]:
long_dis['굴곡도'] = 0
for name in routeNms:

    seq = getStationByRoute[name][getStationByRoute[name]['transYn'] == 'Y'].head(1)
    seq = seq['seq'].values[0]
    seq

    #실제 노선 거리
    df2 = copy.deepcopy(getStationByRoute[name])
    df2['seq'] = df2['seq'].astype(int)
    df2 = df2[df2['seq'] <= int(seq)]
    actual_distance = df2['fullSectDist'].astype(int).sum() / 1000
    actual_distance
    
    # 각 행에 대해 거리 계산
    direct_distance = distance(df2[df2['seq'] == 1], df2[df2['seq'] == int(seq)])

    #굴곡도 계산
    ratio = actual_distance/direct_distance

    idx = long_dis[long_dis['노선명'] == name].index[0]
    long_dis.at[idx,'굴곡도'] = ratio


노선 총 거리 nan값 변경

In [401]:
#nan값 컬럼 대치
keys = long_dis[long_dis['노선총거리(km)'].isna()]['노선명'].values     #nan 값 노선명

for key in keys:
    od_df = temp[temp['노선명'] == key] # od 노선명 필터링 df
    dis_df = getStationByRoute[key] # 역간 거리정보 df 노선명 기준 필터링        
    print(key)

    distance = dis_df['fullSectDist'].astype(int).sum() / 1000
    idx = long_dis[long_dis['노선명'] == key].index[0]
    long_dis.at[idx, '노선총거리(km)'] = distance

1130
1014
110A고려대
110B국민대
111
121
130
144
160
162
2224
2416
500
5513
600
605
651
6516
654
6613
6629
6631
6640A
6640B
6645
6649
6654
6712
672
750B
7739
8551
8773
1114
2234
2235
6635
8761
8772
1133
507
8771


In [403]:
long_dis[long_dis['노선명'] == '143']

,노선명,중장거리비율,노선총거리(km),총승객수,굴곡도
61,143,0.274593,62.0,37402.8618,1.865972


In [404]:
long_dis.to_csv('/Users/taejin/Desktop/데분 학회/급행버스/route_kmeans.csv')